In [1]:
%pip install python-binance pyspark

     |████████████████████████████████| 63 kB 2.0 MB/s eta 0:00:011
     |████████████████████████████████| 102 kB 9.5 MB/s eta 0:00:01
     |████████████████████████████████| 1.2 MB 13.2 MB/s eta 0:00:01
     |████████████████████████████████| 288 kB 41.9 MB/s eta 0:00:01
     |████████████████████████████████| 218 kB 32.2 MB/s eta 0:00:01
     |████████████████████████████████| 203 kB 26.2 MB/s eta 0:00:01
     |████████████████████████████████| 174 kB 44.7 MB/s eta 0:00:01
     |████████████████████████████████| 304 kB 34.9 MB/s eta 0:00:01
     |████████████████████████████████| 763 kB 16.8 MB/s eta 0:00:01
     |████████████████████████████████| 339 kB 44.0 MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [4]:
from binance.client import Client
import configparser
import iceberg_spark
from pyspark.sql.functions import *
import time
spark, jvm, hive_uri, sc = iceberg_spark.start_spark('s3', 'aws', 'hadoop')

# Read config file
config = configparser.ConfigParser()
config.read('config.ini')
api_secret = config['binance']['secret']
api_key = config['binance']['key']

# create client
client = Client(api_key, api_secret, tld='us')


In [5]:
# get current prices from binance
tickers = client.get_all_tickers()

#convert to a spark dataframe, cast data type to decimal, and add date and timestamp columns
tickersDF = spark.createDataFrame(data=tickers)
tickersDF = tickersDF.withColumn("price", col("price").cast("decimal(38,8)"))
tickersDF = tickersDF.withColumn("date", current_date()).withColumn("ts", current_timestamp())


#write to an iceberg table
tickersDF.writeTo('tickers').using('iceberg').partitionedBy('date').createOrReplace()

21/11/18 04:17:21 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties
21/11/18 04:17:35 WARN HadoopTableOperations: Error reading version hint file s3a://brock-dremio/iceberg/warehouse/default/tickers/metadata/version-hint.text
java.io.FileNotFoundException: No such file or directory: s3a://brock-dremio/iceberg/warehouse/default/tickers/metadata/version-hint.text
	at org.apache.hadoop.fs.s3a.S3AFileSystem.s3GetFileStatus(S3AFileSystem.java:2269)
	at org.apache.hadoop.fs.s3a.S3AFileSystem.innerGetFileStatus(S3AFileSystem.java:2163)
	at org.apache.hadoop.fs.s3a.S3AFileSystem.getFileStatus(S3AFileSystem.java:2102)
	at org.apache.hadoop.fs.s3a.S3AFileSystem.open(S3AFileSystem.java:702)
	at org.apache.hadoop.fs.FileSystem.open(FileSystem.java:899)
	at org.apache.iceberg.hadoop.HadoopTableOperations.findVersion(HadoopTableOperations.java:316)
	at org.apache.iceberg.hadoop.HadoopTableOperations.refresh(HadoopTableOperati

In [6]:
spark.table("tickers").show()

+--------------+-------+----------+--------------------+
|         price| symbol|      date|                  ts|
+--------------+-------+----------+--------------------+
|59910.49000000| BTCUSD|2021-11-18|2021-11-18 04:17:...|
| 4286.50000000| ETHUSD|2021-11-18|2021-11-18 04:17:...|
|    0.29700000| XRPUSD|2021-11-18|2021-11-18 04:17:...|
|  589.18000000| BCHUSD|2021-11-18|2021-11-18 04:17:...|
|  222.25000000| LTCUSD|2021-11-18|2021-11-18 04:17:...|
|    1.00060000|USDTUSD|2021-11-18|2021-11-18 04:17:...|
|59871.73000000|BTCUSDT|2021-11-18|2021-11-18 04:17:...|
| 4284.68000000|ETHUSDT|2021-11-18|2021-11-18 04:17:...|
|    0.29691000|XRPUSDT|2021-11-18|2021-11-18 04:17:...|
|  588.56000000|BCHUSDT|2021-11-18|2021-11-18 04:17:...|
|  221.96000000|LTCUSDT|2021-11-18|2021-11-18 04:17:...|
|  580.19280000| BNBUSD|2021-11-18|2021-11-18 04:17:...|
|  579.72350000|BNBUSDT|2021-11-18|2021-11-18 04:17:...|
|    0.07153000| ETHBTC|2021-11-18|2021-11-18 04:17:...|
|    0.00000864| XRPBTC|2021-11

In [7]:
# create funtion to append data
def get_tickers():
    # get current prices from binance
    tickers = client.get_all_tickers()
    
    #convert to a spark dataframe & add date and timestamp columns
    tickersDF = spark.createDataFrame(data=tickers)
    tickersDF = tickersDF.withColumn("price", col("price").cast("decimal(38,8)"))
    tickersDF = tickersDF.withColumn("date", current_date()).withColumn("ts", current_timestamp())
    
    
    #write to an iceberg table
    tickersDF.writeTo('tickers').append()

In [8]:
# Function to repeat a task
def load_tickers(interval):
    while True:
        get_tickers()
        time.sleep(interval)

In [9]:
# call periodic work every 10 seconds
load_tickers(10)

NameError: name 'time' is not defined

# Spark Writes
(https://iceberg.apache.org/spark-writes/) [https://iceberg.apache.org/spark-writes/]

# Writing with DataFrames
[apache Iceberg](https://iceberg.apache.org/spark-writes/#writing-with-dataframes)
<ul>
    <li>df.writeTo(t).create() is equivalent to CREATE TABLE AS SELECT</li>
    <li>df.writeTo(t).replace() is equivalent to REPLACE TABLE AS SELECT</li>
    <li>df.writeTo(t).append() is equivalent to INSERT INTO</li>
    <li>df.writeTo(t).overwritePartitions() is equivalent to dynamic INSERT OVERWRITE</li>
</ul>
